# Stochastic Mesh Laplacian

加载相关的库

In [1]:
import igl
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact

import os
root_folder = os.getcwd()
#root_folder = os.path.join(os.getcwd(), "tutorial")

读取犰狳文件

In [2]:
## Load a mesh in OFF format
v, f = igl.read_triangle_mesh(os.path.join(root_folder, "data", "Armadillo4.off"))
print("网格的平均边长：%f" % igl.avg_edge_length(v, f))

## Print the vertices and faces matrices 
print("Vertices: ", len(v))
print("Faces: ", len(f))

网格的平均边长：0.852035
Vertices:  69191
Faces:  138378


## 网格的多尺度表示LB算子

In [56]:
l = -igl.cotmatrix(v, f)
m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)

k = 10
d, u = sp.sparse.linalg.eigsh(l, k, m, sigma=0, which="LM")

# 渲染原始的图
n = igl.per_vertex_normals(v, f)*0.5+0.5
c = np.linalg.norm(n, axis=1)
p = plot(v, f, c, shading={"wireframe": False}, return_plot=True)


p_k = u @ (u.T @ m.T @ v)
print(p_k.shape)
# p_k = u.dot(u.T).dot(m.T).dot(p)
n = igl.per_vertex_normals(p_k, f)*0.5+0.5
c = np.linalg.norm(n, axis=1)
p.update_object(vertices=p_k, colors=c)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0100498…

(69191, 3)


## 计算随机网格的拉普拉斯矩阵

网格顶点的显著性值的计算

In [6]:
import time
start = time.clock()
adj_list = igl.adjacency_list(f)
average_length = igl.avg_edge_length(v, f)
alpha = []
D = []
A = igl.adjacency_matrix(f)
W = A.astype(float)    # 转换数据类型
for i in range(len(adj_list)):
    n = len(adj_list[i])
    D.append(n)
    ring_length = 0
    for j in range(n):
        dist = np.linalg.norm(v[i] - v[adj_list[i][j]])
        ring_length += dist
        W[i, adj_list[i][j]] = 1 / (dist * dist)
    
    alpha_i = (average_length * n) / ring_length + 1
    alpha.append(alpha_i)
    
D = sp.sparse.diags(D)   # 顶点的入度矩阵
D = D.astype(float)
L = D - W  # 网格的拉普拉斯矩阵
L_abs = abs(L)

# 矩阵L的归一化
def normalization(mat):
    '''
    mat:numpy.matrix
    对输入的数据进行基于行的归一化
    '''
    mat_row_sum = mat.sum(axis = 1, dtype='float')
    mat_row_sum = sp.sparse.diags(1 / mat_row_sum.A1.ravel())
    mat_norm = mat @ mat_row_sum
    
    return mat_norm

LL = normalization(L_abs)  # 网格的随机拉普拉斯矩阵
k = 5  # 网格的尺度
lamda = 0.001 # 

alpha = sp.sparse.diags(alpha)

import numpy.matlib
I = np.matlib.zeros((LL.shape[0], 1))

# 计算每个顶点的显著性值
F = LL
for i in range(2, k + 1):
    F_next = F @ L_abs
    F_next = normalization(F_next)
    # F_next = np.where(F_next >= lamda, F_next, 0)
    F_nonzero = F_next.nonzero()
    row = F_nonzero[0]
    col = F_nonzero[1]
    for i in range(len(row)):
            if F_next[row[i], col[i]] < lamda:
                F_next[row[i], col[i]] = 0
    F_next = F_next @ alpha
    F_next = normalization(F_next)
    D = abs(F_next - F)
    D = D.sum(axis = 1, dtype = "float")
    I = I + D
    F = F_next

'''
P = F @ v
# 尺度变化之后的渲染图
n = igl.per_vertex_normals(P, f)*0.5+0.5
c = np.linalg.norm(n, axis=1)
p = plot(P, f, c, shading={"wireframe": False}, return_plot=True)
'''

I = np.log(I)
end = time.clock()
runtime = end - start
print(I)
print(runtime)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-6.817447…

[[-0.17257498]
 [ 0.17353634]
 [ 0.08367856]
 ..., 
 [ 0.31660952]
 [-0.00795572]
 [ 0.12659732]]
882.6761415999999


计算网格的显著性顶点

In [26]:
I_array = I.A1
I_max = np.max(I_array)
I_std = np.std(I_array)
if I_std > 1 :   
    I_max = I_max / I_std

threshold = 0.25 * I_max  # 全局显著性的设置
selected_points = []
for i in range(len(adj_list)):
    add = True
    for j in adj_list[i]:
        if I_array[j] > I_array[i]:
            add = False
            break
    if add and I_array[i] > threshold:
        selected_points.append(v[i])

selected_points = np.array(selected_points)
n = igl.per_vertex_normals(v, f)*0.5+0.5
c = np.linalg.norm(n, axis=1)
p = plot(v, f, c, shading={"wireframe": False}, return_plot=True)
p.add_points(selected_points, shading={"point_color": "red", "point_size": 12})
print(len(selected_points))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0100498…

277
